In [265]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [266]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"SSRM"}, 
             {'symbol':"LRN"}, 
             {'symbol':"UNFI", 'basis':22.200}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':100.482}, 
             {'symbol':"EZPW", 'basis':15.460}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':62.588}, 
             {'symbol':"ITRN", 'basis':35.710}, 
             {'symbol':"CRDO", 'basis':0.010}, 
             {'symbol':"PYPL", 'basis':68.738}, 
             {'symbol':"ALL", 'basis':195.419}, 
             {'symbol':"LC", 'basis':9.384}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':0.010}, 
             {'symbol':"CCL", 'basis':11.617}, 
             {'symbol':"AGX", 'basis':0.010}, 
             {'symbol':"POWL", 'basis':181.166}, 
             {'symbol':"PPC", 'basis':45.089}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE"}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':73.160}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA", 'basis':101.620}, 
             {'symbol':"TWLO", 'basis':87.178}, 
             {'symbol':"APP", 'basis':221.323}, 
             {'symbol':"TMUS", 'basis':225.525}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':40.612}, 
             {'symbol':"CAAP", 'basis':21.570}
             ]

# Brokerage
brokerage_funds = [{'symbol':"SSRM"}, 
                   {'symbol':'BCS', 'basis':18.580}, # UK - Financials
                   {'symbol':'PUK'}, # Hong Kong - Financials
                   {'symbol':'FINV'}, # China - Financials
                   {'symbol':'PSIX'}, # U.S. - Industrials
                   {'symbol':'NGD'}, # Canada - Materials (Gold)
                   {'symbol':'GFI'}, # South Africa - Materials (Gold)
                   {'symbol':"CRDO", 'basis':17.173}, 
                   {'symbol':"COMM"}, 
                   {'symbol':'BKTI'}, # United States - Communication Equipment
                   {'symbol':"UNFI", 'basis':22.950}, 
                   {'symbol':"EZPW", 'basis':17.715}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"WFC", 'basis':68.586}, 
                   {'symbol':"PYPL", 'basis':69.126}, 
                   {'symbol':"CLS", 'basis':74.924}, 
                   {'symbol':"CCL", 'basis':17.866}, 
                   {'symbol':"SYF"}, 
                   {'symbol':"ATGE", 'basis':86.385}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"OKTA", 'basis':104.950}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"UBER"},
                   #{'symbol':"CAAP"},
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':95.19},
                   {'symbol':"QUBT"},
                   {'symbol':"RGTI"}
                   ]

In [267]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.95):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [268]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=1, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [269]:
brokerage_df[brokerage_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
1,BCS,18.580,18.61,16.09,45.10,16.46,9.08,0.643,0.0227,Sell
7,CRDO,17.173,93.49,58.24,6.32,62.69,322.38,23.290,0.0000,Sell
10,UNFI,22.950,23.32,24.43,24.43,25.82,inf,0.849,0.0000,Hold
11,EZPW,17.715,13.62,14.69,23.84,15.26,11.45,0.978,0.0000,Hold
12,ARQT,15.996,14.02,13.51,13.51,14.54,inf,11.490,0.0000,Hold
13,WFC,68.586,79.50,69.61,97.58,71.88,14.30,1.592,0.0201,Sell
14,PYPL,69.126,74.32,66.21,56.89,68.51,16.55,3.536,0.0000,Sell
15,CLS,74.924,152.67,96.65,40.71,103.08,42.76,11.280,0.0000,Sell
16,CCL,17.866,28.10,20.77,22.67,21.75,14.27,3.693,0.0000,Sell
18,ATGE,86.385,127.27,114.99,89.05,120.47,21.74,3.222,0.0000,Sell


In [270]:
brokerage_df[brokerage_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,SSRM,0.0,12.31,11.09,14.82,11.65,30.02,0.788,0.0000,Sell
2,PUK,0.0,25.00,21.84,27.78,22.30,15.00,1.851,0.0184,Sell
3,FINV,0.0,9.47,8.44,20.00,8.88,6.86,1.121,0.0290,Sell
4,PSIX,0.0,64.68,37.78,20.41,41.25,18.91,18.210,0.0000,Sell
5,NGD,0.0,4.75,3.99,2.84,4.18,26.39,3.638,0.0000,Sell
6,GFI,0.0,22.87,22.29,16.59,23.13,16.57,3.935,0.0238,Hold
8,COMM,0.0,8.20,5.00,5.00,5.43,inf,30.360,0.0000,Sell
9,BKTI,0.0,53.54,44.13,28.08,48.69,20.51,6.080,0.0000,Sell
17,SYF,0.0,66.74,52.93,101.31,55.33,9.07,1.641,0.0181,Sell
22,UBER,0.0,93.28,80.53,46.23,83.76,16.03,8.710,0.0000,Sell


In [271]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=.95, vwap_days=63)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [272]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
2,UNFI,22.200,23.32,23.21,23.21,25.82,inf,0.849,0.0000,Hold
4,EAT,100.482,180.30,142.18,37.93,157.85,25.08,30.900,0.0000,Sell
5,EZPW,15.460,13.62,13.96,23.84,15.26,11.45,0.978,0.0000,Hold
6,ARQT,15.932,14.02,12.83,12.83,14.54,inf,11.490,0.0000,Hold
7,WFC,62.588,80.06,66.30,98.26,72.04,14.30,1.592,0.0201,Sell
8,ITRN,35.710,38.73,33.32,31.00,36.38,13.63,3.927,0.0531,Sell
9,CRDO,0.010,92.59,55.88,6.26,63.28,322.38,23.290,0.0000,Sell
10,PYPL,68.738,74.32,62.90,56.89,68.51,16.55,3.536,0.0000,Sell
11,ALL,195.419,201.24,185.38,200.46,200.66,13.37,2.585,0.0204,Sell
12,LC,9.384,12.03,9.57,13.49,10.57,27.20,1.002,0.0000,Sell


In [273]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,SSRM,0.0,12.74,10.55,15.34,11.66,30.02,0.788,0.0000,Sell
1,LRN,0.0,145.23,134.04,84.19,147.63,22.82,4.472,0.0000,Hold
3,MFC,0.0,31.99,28.62,35.70,30.99,16.58,1.661,0.0402,Sell
13,QTWO,0.0,93.99,77.73,77.73,85.17,inf,10.740,0.0000,Sell
19,SYF,0.0,66.74,50.28,101.31,55.33,9.07,1.641,0.0181,Sell
20,ATGE,0.0,127.27,109.24,89.05,120.47,21.74,3.222,0.0000,Sell
21,BRK-B,0.0,485.68,476.81,624.65,514.35,12.95,1.601,0.0000,Hold
22,SFM,0.0,164.64,150.71,45.45,165.95,36.52,12.320,0.0000,Hold
24,BLBD,0.0,43.16,34.66,25.83,38.01,13.39,7.150,0.0000,Sell
25,GM,0.0,49.68,43.77,128.21,47.64,6.94,0.742,0.0121,Sell


In [274]:
# Research new investments
watch_list = [{'symbol':'BSAC'}, # Chile - Latin America - Financials
              {'symbol':'KARO'}, # Singapore - Technology - SaaS
              {'symbol':'QFIN'}, # China - FinTech
              {'symbol':'TSM'}, # Taiwan - Semiconductors
              {'symbol':'CCU'}, # Chile - Consumer Staples - beverage distributor
              {'symbol':'ABEV'}, # Brazil - Financials - but distributes beers?
              {'symbol':'TIMB'}, # Brazil - Communications - Telecom (mobile networking)
              {'symbol':'ASR'}, # Mexico - Industrials - Airport operations in South America
              {'symbol':'RYAAY'}, # Ireland - Industrials - budget airline
              {'symbol':'AMX'}, # Mexico - Communications - mobile network
              {'symbol':'ERIC'}, # Sweden - Communications
              {'symbol':'STX'} # Ireland - Technology
             ] 
watch_table = build_analysis_table(watch_list, api_key, margin_of_safety=1, vwap_days=63)
internationals = build_analysis_table(watch_list, api_key, margin_of_safety=0.95, vwap_days=126)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

internationals_df = pd.DataFrame(internationals, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [275]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,25.20,23.63,28.74,24.25,10.73,2.458,0.0549,Sell
1,KARO,0,47.74,47.13,17.90,50.24,28.42,8.580,0.0226,Hold
2,QFIN,0,43.36,39.94,71.05,41.99,6.86,1.862,0.0292,Sell
3,TSM,0,226.46,173.36,92.59,179.84,26.89,7.630,0.0144,Sell
4,CCU,0,12.89,14.16,17.11,14.64,13.43,1.450,0.0354,Hold
5,ABEV,0,2.40,2.39,2.49,2.46,14.75,2.168,0.0647,Hold
6,TIMB,0,20.12,16.92,20.67,17.34,15.20,2.139,0.0794,Sell
7,ASR,0,319.12,305.62,285.51,314.42,13.09,3.273,0.0787,Sell
8,RYAAY,0,56.99,50.00,42.48,51.31,16.76,3.683,0.0164,Sell
9,AMX,0,17.93,16.25,10.80,16.68,35.73,2.645,0.0312,Sell


In [276]:
internationals_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,BSAC,0,25.20,21.68,28.74,23.37,10.73,2.458,0.0549,Sell
1,KARO,0,48.98,43.69,18.37,48.73,28.42,8.580,0.0226,Sell
2,QFIN,0,43.36,38.57,71.05,42.74,6.86,1.862,0.0292,Sell
3,TSM,0,226.46,173.66,92.59,189.69,26.89,7.630,0.0144,Sell
4,CCU,0,12.91,13.03,17.13,14.13,13.43,1.450,0.0354,Hold
5,ABEV,0,2.40,2.04,2.49,2.21,14.75,2.168,0.0647,Sell
6,TIMB,0,20.12,14.32,20.67,15.41,15.20,2.139,0.0794,Sell
7,ASR,0,319.12,276.68,285.51,300.03,13.09,3.273,0.0787,Sell
8,RYAAY,0,57.67,45.34,42.99,48.96,16.76,3.683,0.0164,Sell
9,AMX,0,17.93,14.71,10.80,15.88,35.73,2.645,0.0312,Sell


In [277]:
# Research new investments
mag7_test = [{'symbol':'MSFT'}, 
             {'symbol':'AAPL'}, 
             {'symbol':'NVDA'}, 
             {'symbol':'AMZN'}, 
             {'symbol':'GOOG'}, 
             {'symbol':'GOOGL'}, 
             {'symbol':'META'}, 
             {'symbol':'BRK-B'}, 
             {'symbol':'TSLA'}
             ] 
mag7_test_table = build_analysis_table(mag7_test, api_key, vwap_days=63)

# Create watch list data frame
mag7_df = pd.DataFrame(mag7_test_table, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [278]:
mag7_test_table

[['MSFT', 0, 497.41, 379.22, 139.05, 431.49, 38.33, 11.45, 0.0067, 'Sell'],
 ['AAPL', 0, 205.17, 176.66, 31.99, 204.02, 31.37, 44.96, 0.0052, 'Sell'],
 ['NVDA', 0, 157.99, 106.88, 19.18, 124.25, 50.72, 45.88, 0.0003, 'Sell'],
 ['AMZN', 0, 219.39, 172.36, 76.53, 198.77, 36.37, 7.75, 0, 'Sell'],
 ['GOOG', 0, 177.39, 145.98, 93.07, 167.95, 19.87, 6.27, 0.0047, 'Sell'],
 ['GOOGL', 0, 178.53, 143.92, 93.53, 165.75, 19.93, 6.27, 0.0047, 'Sell'],
 ['META', 0, 733.63, 527.0, 254.16, 608.53, 28.66, 9.97, 0.0029, 'Sell'],
 ['BRK-B', 0, 485.3, 451.1, 624.16, 513.51, 12.95, 1.601, 0, 'Hold'],
 ['TSLA', 0, 323.63, 257.26, 37.29, 305.35, 185.02, 13.96, 0, 'Sell']]

In [279]:
mag7_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,497.41,379.22,139.05,431.49,38.33,11.450,0.0067,Sell
1,AAPL,0,205.17,176.66,31.99,204.02,31.37,44.960,0.0052,Sell
2,NVDA,0,157.99,106.88,19.18,124.25,50.72,45.880,0.0003,Sell
3,AMZN,0,219.39,172.36,76.53,198.77,36.37,7.750,0.0000,Sell
4,GOOG,0,177.39,145.98,93.07,167.95,19.87,6.270,0.0047,Sell
5,GOOGL,0,178.53,143.92,93.53,165.75,19.93,6.270,0.0047,Sell
6,META,0,733.63,527.00,254.16,608.53,28.66,9.970,0.0029,Sell
7,BRK-B,0,485.30,451.10,624.16,513.51,12.95,1.601,0.0000,Hold
8,TSLA,0,323.63,257.26,37.29,305.35,185.02,13.960,0.0000,Sell


In [280]:
# Machine Learning

In [281]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [282]:
# Feature selection and normalization

In [283]:
# Build model

In [284]:
# Train model

In [285]:
# Evaluate model

In [286]:
# Refine model through hyperparameter tuning